In [1]:
import torch

# Local GPU Test

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.5.1
True
NVIDIA GeForce RTX 4080


In [2]:
import objaverse

uids = objaverse.load_uids() # each object has a unique ID (uid), then load it
print("all of objects length :", len(uids))

lvis_annotations = objaverse.load_lvis_annotations()

num_of_objects = 10
filtered_lvis_annotations = {key: value for key, value in lvis_annotations.items() if len(value) >= num_of_objects}
print(f"the number of classes having items more than {num_of_objects} : {len(filtered_lvis_annotations)}")

all of objects length : 798759
the number of classes having items more than 10 : 1061


In [3]:
my_datasets = {}

for key, value in filtered_lvis_annotations.items():
    sampled_uids = value[:num_of_objects]
    my_datasets[key] = sampled_uids
    print(f"{key} has {len(sampled_uids)} items : {sampled_uids}")
print(f"This dataset consists of {len(my_datasets)} classes.")

Bible has 10 items : ['ee7660359dda4c90990cb7440211b873', '3e5393bd35e34531a576e22d45611060', 'f5bf29193be44caaad2df1db3db44871', 'adde1e2669d14286843bc9f1cff9ae45', 'f0ed46d723dd441bb592d715db241097', '9d3007d40ca84bb7baec1f1a04e0f08d', '797e7c7f61104b6785c3bd0fdb99c74d', '149d47c6c27648a6af26d6c912c03826', '8257772b0e2f408ba269264855dfea00', 'e7b30aa0ad754744bb5344bc86bf794a']
CD_player has 10 items : ['b4af2a721b1645ddb77168d5e8fe4882', '830b1864b50e4f7e94c7f5cee5b64b43', '639a89472cfa4794b848175bbf652136', '00a66884dacb498b85701acd1cc77c5e', 'dbf0dbcf4229429088b0a984e8f05666', '6914d1e81eeb4377b61af0e5321ede5b', '73e76edb90914783b0fd732ea53b2e46', '9749f5ba1221476993376a3cb8fee1a5', 'f6c8386e76e54918b551ebdd276479dd', 'a4bc4accb20b43ffa728a7a85b7dfed7']
Christmas_tree has 10 items : ['e283aa7835664a74a4ad39afb26d2ef3', 'ae0c3996ee0345aaa8bbb717db7ffb25', 'fe2b4a2708334ff880f62a947cfd3d50', '2da008c410be42d188a0c95aa1bca05e', '16fbadc669454c26aa0927445b3c6ffd', '0b1b92d6d5584284b59a

In [ ]:
# once you download the datasets, you don't need to run it again.

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

count = 1
for key, value in my_datasets.items():
    print(f"\nDownloading {key}... ({count} / {len(my_datasets)})")
    objaverse.load_objects(value, 1)
    print(f"Downloading {key} done.\n")
    count += 1

print("All downloading are done.")


Downloaded 1 / 10 objects
Downloaded 2 / 10 objects
Downloaded 3 / 10 objects
Downloaded 4 / 10 objects
Downloaded 5 / 10 objects
Downloaded 6 / 10 objects
Downloaded 7 / 10 objects
Downloaded 8 / 10 objects
Downloaded 9 / 10 objects
Downloaded 10 / 10 objects


Downloaded 1 / 10 objects
Downloaded 2 / 10 objects
Downloaded 3 / 10 objects
Downloaded 4 / 10 objects
Downloaded 5 / 10 objects
Downloaded 6 / 10 objects
Downloaded 7 / 10 objects
Downloaded 8 / 10 objects
Downloaded 9 / 10 objects
Downloaded 10 / 10 objects


Downloaded 1 / 10 objects


In [11]:
import json
import trimesh
import gzip
import pyrender
import numpy as np
import os
from PIL import Image

class Renderer:
    def __init__(self, path_file, output_path):
        self.my_datasets = my_datasets # key : a class, value : a list of uids
        self.base_path = os.path.join("D:\Studies\objaverse", "hf-objaverse-v1")
        with gzip.open(path_file, 'rt') as f:
            self.object_paths = json.load(f)
        self.output_path = output_path
        
    def load_mesh(self, uid):
        # load 3D .glb mesh from its UID
        mesh_path = os.path.join(self.base_path, self.object_paths[uid])
        if not mesh_path or not os.path.exists(mesh_path):
            raise FileNotFoundError(f"Model file not found for UID: {uid}")
        mesh = trimesh.load(mesh_path, force='mesh')
        
        # normalize the mesh to fit inside a 2*2*2 cube centered at (0,0,0)
        centroid = mesh.bounds.mean(axis=0)
        mesh.apply_translation(-centroid)
        
        max_extent = mesh.extents.max()
        scale_factor = 2.0 / max_extent
        mesh.apply_scale(scale_factor)
        
        return mesh
    
    def render_mesh(self, 
                    mesh, 
                    camera_pos=[
                        (1, 0.5, 1),
                        (0, 0.5, 1),
                        (-1, 0.5, 1),
                        (1, 0.5, -1),
                        (0, 0.5, -1),
                        (-1, 0.5, -1)
                    ], 
                    distance=2.5, 
                    image_size=(512, 512)):
        
        scene = pyrender.Scene()
        mesh = pyrender.Mesh.from_trimesh(mesh)
        scene.add(mesh)
        
        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
        
        images = []
        
        for position in camera_pos:
            camera_position = np.array(position)
    
            target = np.array([0, 0, 0])
            up = np.array([0, 1, 0])
    
            forward = target - camera_position
            forward /= np.linalg.norm(forward)
    
            right = np.cross(up, forward)
            right /= np.linalg.norm(right)
    
            up_corrected = np.cross(forward, right)
            
            rotation_matrix = np.vstack([right, up_corrected, -forward]).T
    
            camera_pose = np.eye(4)
            camera_pose[:3, :3] = rotation_matrix
            camera_pose[:3, 3] = -rotation_matrix @ camera_position
    
            scene.add(camera, pose=camera_pose)
            scene.add(light, pose=camera_pose)
    
            r = pyrender.OffscreenRenderer(*image_size)
            color, _ = r.render(scene)
            images.append(Image.fromarray(color))
    
            scene.remove_node(scene.get_nodes(name=camera.name)[0])
            scene.remove_node(scene.get_nodes(name=light.name)[0])
        
        return images
    
    def process_dataset(self):
        for cls, uids in self.my_datasets.items():
            class_dir = os.path.join(self.output_path, cls)
            os.makedirs(class_dir, exist_ok=True)
            
            for uid in uids:
                try:
                    mesh = self.load_mesh(uid)
                    images = self.render_mesh(mesh)
                    
                    for i, img in enumerate(images):
                        img_path = os.path.join(class_dir, f"{uid}_{i}.png") # class path + img name
                        img.save(img_path)
                        print(f"Saved {img_path}")
                        
                except Exception as e:
                    print(f"Error processing UID {uid}: {e}")
        
        print("Rendering completed!")

In [12]:
renderer = Renderer("D:\Studies\objaverse\hf-objaverse-v1\object-paths.json.gz", "D:\Studies\output")
renderer.process_dataset()

Error processing UID 5b1878a78d2849a494a3e8c95238d727: Expected a Trimesh or a list, got a <class 'trimesh.scene.scene.Scene'>
Error processing UID 5125528b0ec043b98124ceaa9e44168f: Expected a Trimesh or a list, got a <class 'trimesh.scene.scene.Scene'>
Error processing UID b4a2c7380da24adf8ac7777870088983: Model file not found for UID: b4a2c7380da24adf8ac7777870088983
Error processing UID 281891d0dd6a4affbda7530bed83f846: Expected a Trimesh or a list, got a <class 'trimesh.scene.scene.Scene'>
Error processing UID d5d882096fda4248bb5ff5dc6c37300e: Model file not found for UID: d5d882096fda4248bb5ff5dc6c37300e
Error processing UID a17145104f5e4492b48c8dfd2c23677a: Expected a Trimesh or a list, got a <class 'trimesh.scene.scene.Scene'>
Error processing UID 8f4af0f494fb4a01924d53ee7c34fa35: Model file not found for UID: 8f4af0f494fb4a01924d53ee7c34fa35
Error processing UID b2a917737eab4866a8fda30fded72d1c: Model file not found for UID: b2a917737eab4866a8fda30fded72d1c
Error processing UID

KeyboardInterrupt: 